In [7]:
import json
import cv2
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
from math import sqrt
import math
import copy
from scipy.spatial import distance
from pathlib import Path
import scipy

In [32]:
def get_character_plane(data, order=2):
    if order == 1:
        # best-fit linear plane z = a*x + b*y + c, where a, b, c are the cofficients that need to find
        # a = C[0], b = C[1], c = C[2]
        A = np.c_[data[:,0], data[:,1], np.ones(data.shape[0])]
        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])    # coefficients
        return [C[0], C[1], -1, C[2]]
    elif order == 2:
        # best-fit quadratic curve z = a + b*x + c*y + d*x*y + e*x*x + f*y*y
        # a = C[0], b = C[1], c= C[2], d = C[3], e = C[4], f = C[5]
        A = np.c_[np.ones(data.shape[0]), data[:,:2], np.prod(data[:,:2], axis=1), data[:,:2]**2]
        C,_,_,_ = scipy.linalg.lstsq(A, data[:,2])
        return [C[0], C[1], C[2], -1, C[3], C[4], C[5]]


def calc_sin_phi(a, b, c):
    return sqrt((a*a + b*b) / (a*a + b*b + c*c))

def calc_cos_phi(a, b, c):
    return c / sqrt(a*a + b*b + c*c)

def calc_u1(a, b, c):
    return b / sqrt(a*a + b*b)


def calc_u2(a, b, c):
    return -a / sqrt(a*a + b*b)

def get_rotation_matrix(plane_coff):
    a, b, c, d = plane_coff
    cos_phi = calc_cos_phi(a, b, c)
    sin_phi = calc_sin_phi(a, b, c)
    u1 = calc_u1(a, b, c)
    u2 = calc_u2(a, b, c)
    rot_matrix = np.array([
        [cos_phi + u1 * u1 * (1 - cos_phi)  , u1 * u2 * (1 - cos_phi)           , u2 * sin_phi  ,  0],
        [u1 * u2 * (1 - cos_phi)            , cos_phi + u2 * u2 * (1 - cos_phi) , -u1 * sin_phi ,  0],
        [-u2 * sin_phi                      , u1 * sin_phi                      ,      cos_phi  , 0],
        [0                                  , 0                                 , 0             ,  1]])
    return rot_matrix

def get_trans_matrix(plane_coff, transform_matrix):
    # t_inv = np.linalg.inv(transform_matrix)
    # new_plane_coff = np.dot(plane_coff, t_inv)
    t_matrix = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, -plane_coff[3]], [0, 0, 0, 1]])
    return t_matrix

def get_transform_matrix(plane_coff):
    rot_matrix = get_rotation_matrix(plane_coff)
    #print(rot_matrix)
    trans_matrix = get_trans_matrix(plane_coff, rot_matrix)
    #print(trans_matrix)
    t_r_matrix = np.matmul(rot_matrix, trans_matrix)
    #print(t_r_matrix)
    return t_r_matrix

def do_transform(np_pc_points, transform_matrix):
    ones = np.ones((np_pc_points.shape[0], 1))
    homo_np_pc_points = np.c_[ np_pc_points, ones]
    homo_pc_3d = transform_matrix@(homo_np_pc_points.T)
    homo_pc_3d = homo_pc_3d.T[:, :3]
    
    mean_vector = np.mean(homo_pc_3d, axis=0)
    if mean_vector[2] < 0:
        # upsiding if front face is down side
        homo_pc_3d[:, 2] = - homo_pc_3d[:, 2]
        homo_pc_3d[:, 0] = - homo_pc_3d[:, 0]
        mean_vector = np.mean(homo_pc_3d, axis=0)
    mean_vector[2] = 0
    homo_pc_3d = homo_pc_3d - mean_vector
    return homo_pc_3d

In [33]:
def read_stl_file(pc_path):
    pc_points = o3d.io.read_triangle_mesh(str(pc_path))
    np_pc_points = np.asarray(pc_points.vertices)
    return pc_points, np_pc_points

In [34]:
woodblock_file_path = "/mnt/hdd/thuonglc/mocban/data_synthesis/woodblock-gt-depth-gen/data/models_3d/08360_1_point_on_surface.stl"

In [35]:
pc_points, np_pc_points = read_stl_file(woodblock_file_path)
C = get_character_plane(np_pc_points, order=2)

In [37]:
C

[0.9791627857780628,
 0.011456807750208428,
 -0.011223323458816228,
 -1,
 7.70934791409991e-05,
 6.130547467097895e-06,
 0.00015713562515484698]

In [73]:
C[5]

6.130547467097895e-06

In [74]:
def calculate_gradient(C, points):
    gradient_of_x = C[1] + C[4] * points[:, 1] + 2*C[5]*points[:, 0]
    gradient_of_y = C[2] + C[4] * points[:, 0] + 2*C[6]*points[:, 1]
    minus_ones = -np.ones_like(gradient_of_x)
    # length = np.sqrt(np.power(gradient_of_x, 2) + np.power(gradient_of_y, 2)  + 1)
    # if length == 0:
    #     length += 1e-8 
    gradient = np.c_[gradient_of_x, gradient_of_y, minus_ones]
    print(gradient[0])
    return np.mean(gradient, axis=0)   

In [75]:
limit_bounding  = np.asarray([[ 112.30486134, 127.21270628],
                [-117.05013116, -104.44313653]
               ]).astype(np.int16)

x = np.arange(limit_bounding[0][0], limit_bounding[0][1])
y = np.arange(limit_bounding[1][0], limit_bounding[1][1])
# full coordinate arrays
xx, yy = np.meshgrid(x, y)
out = np.column_stack((xx.ravel(),yy.ravel()))

In [78]:
normal_vec = calculate_gradient(C, out)

[ 0.00381011 -0.03935859 -1.        ]


In [79]:
normal_vec

array([ 0.0043585 , -0.03693331, -1.        ])

-0.009006590612850347